# 量子相位估计算法

[![](https://gitee.com/mindspore/docs/raw/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/docs/mindquantum/docs/source_zh_cn/quantum_phase_estimation.ipynb)

## 概述

量子相位估计算法(Quantum Phase Estimation Algorithm，简称QPE)，是很多量子算法的关键。假设一个幺正算符 $U$，这个幺正算符作用在其本征态 $|u\rangle$ 上会出现一个相位 $e^{2\pi i \varphi}$，现在我们假设 $U$ 算符的本征值未知，也就是 $\varphi$ 未知，但是 $U$ 算符和本征态 $|u\rangle$ 已知，相位估计算法的作用就是对这个相位 $\varphi$ 进行估计。

![quantum phase estimation](https://gitee.com/mindspore/docs/raw/master/docs/mindquantum/docs/source_zh_cn/images/quantum_phase_estimation.png)

## 算法解析

相位估计算法的实现需要两个寄存器(register)，第一个寄存器包含$t$个初始在 $|0\rangle$ 的量子比特，比特数和最后相位估计的结果的精度和算法的成功概率相关；第二个寄存器初始化在幺正算符 $U$ 的本征态 $|u\rangle$ 上。相位估计算法主要分为三步：

1. 对第一个寄存器的所有量子比特进行``Hadamard``门操作，对第二寄存器连续进行``控制U``门操作，其中U门的幂次依次为 $2^0, 2^1,...,2^{t-1}$，控制比特依次为 $q_{t-1}, q_{t-2},..., q_{1}, q_{0}$。这时第一寄存器中的态就会变为

$$
|\psi_1\rangle=\frac{1}{2^{t/2}}\left(|0\rangle+e^{i2\pi 2^{t-1}\varphi}|1\rangle\right)\left(|0\rangle+e^{i2\pi2^{t-2}\varphi}|1\rangle\right)...\left(|0\rangle+e^{i2\pi 2^{0}\varphi}|1\rangle\right) = \frac{1}{2^{t/2}}\sum_{k=0}^{2^t-1}e^{i2\pi\varphi k}|k\rangle
$$

其中k为直积态的十进制表示，比如 $k=0$ 表示第一寄存器中t个比特全部在基态 $|00...00\rangle$,  $k=2$ 表示 $|00...10\rangle$，以此类推。

2. 对第一寄存器的进行量子傅里叶变换的逆变换(Inverse Quantum Fourier Transform)，在线路中表示成 $QFT^\dagger$, 对 $|\psi_1\rangle$ 进行逆量子傅里叶变换可得 $|\psi_2\rangle$

$$
|\psi_2\rangle=QFT^\dagger|\psi_1\rangle =\frac{1}{2^t}\sum_{x=0}^{2^t-1}a_x|x\rangle
$$

其中

$$
a_x=\sum_{k=0}^{2^t-1}e^{2\pi i k(\varphi-x/2^t)}
$$

为本征基矢 $|x\rangle$ ($x=0.1,...,2^t$) 对应的概率幅 。由上式可得，当 $2^t\varphi$ 为整数，且满足 $x=2^t\varphi$ 时，概率幅取最大值1，此时第一寄存器的末态可以精确反映 $\varphi$；当 $2^t\varphi$ 不是整数时，$x$ 为 $\varphi$ 的估计，且$t$越大，估计精度越高。

3. 对第一寄存器的量子比特进行测量，得到第一寄存器的末态 $f=\sum_{x}^{2^t-1}a_x|x\rangle$, $x=0,1,...,2^t$；从中找到最大的概率幅 $a_{max}$，其对应的本征基矢 $|x\rangle$ 中的 $x$ 在除以 $2^t$ 即为相位的估计值。

## QPE代码实现

下面用一个实例来演示如何在MindQuantum实现相位估计算法，选择 ``T``门作为进行估计的幺正算符，由定义

$$
T|1\rangle=e^{i\pi/4}|1\rangle
$$

可知需要估计的相位角为 $\varphi=\frac{1}{8}$。

首先导入相关依赖。

In [1]:
from mindquantum import Circuit
from mindquantum import Simulator
from mindquantum import UN, PhaseShift, qft, H, X, BARRIER
import numpy as np

``Mindquantum.UN`` 可以指定量子门，目标比特和控制比特，从而在线路中搭建门操作。因为我们已知 $\varphi=1/8$，当 $t=3$ 时可令 $2^t\varphi$ 为整数，所以第一寄存器只需要3个比特即可准确估计；又已知 ``T`` 门的本征态为 $|1\rangle$，所以第二寄存器选择一个比特，即：我们需要搭建4比特线路，前 $q_0, q_1, q_2$ 比特用于估计，属于第一寄存器；$q_3$ 属于第二寄存器用于传入 $T$ 算符的本征态。

利用 ``UN`` 对 $q_0, q_1, q_2$ 进行 ``Hadamard`` 门操作， 用 ``X`` 门对 $q_3$ 进行翻转，得到 ``T`` 门的本征态 $|1\rangle$。

In [2]:
# pylint: disable=W0104
n = 3
c = Circuit()
c += UN(H, n)
c += X.on(n)
c

q0: ──H──
         
q1: ──H──
         
q2: ──H──
         
q3: ──X──

以 $q_3$ 为目标比特，添加 ``控制PhaseShift``门。

In [3]:
# pylint: disable=W0104
for i in range(n):
    c += PhaseShift({'phi': 2**i}).on(n, n-i-1)
c

q0: ──H────────────────────────────────●──────
                                       │      
q1: ──H───────────────────●────────────┼──────
                          │            │      
q2: ──H───────●───────────┼────────────┼──────
              │           │            │      
q3: ──X────PS(phi)────PS(2*phi)────PS(4*phi)──

对第一寄存器比特进行逆傅里叶变换。

In [4]:
# pylint: disable=W0104
c += BARRIER
c += qft(range(n)).hermitian()
c

q0: ──H────────────────────────────────●────────@──────────────────────────PS(-π/4)────PS(-π/2)────H──
                                       │        │                             │           │           
q1: ──H───────────────────●────────────┼────────┼─────────PS(-π/2)────H───────┼───────────●───────────
                          │            │        │            │                │                       
q2: ──H───────●───────────┼────────────┼────────@────H───────●────────────────●───────────────────────
              │           │            │                                                              
q3: ──X────PS(phi)────PS(2*phi)────PS(4*phi)──────────────────────────────────────────────────────────

选择后端、传入总比特数创建模拟器，将 $\varphi$ 值传入并进行演化，得到末态。

In [5]:
# pylint: disable=W0104
from mindquantum import Measure
sim = Simulator('projectq', c.n_qubits)
phi = 0.125
sim.apply_circuit(c, {'phi': 2*np.pi*phi})
qs = sim.get_qs()
print(sim.get_qs(ket=True))
res = sim.sampling(UN(Measure(), c.n_qubits), shots=100)
res

1¦1100⟩


shots: 100
Keys: q3 q2 q1 q0│0.00     0.2         0.4         0.6         0.8         1.0
─────────────────┼───────────┴───────────┴───────────┴───────────┴───────────┴
             1100│▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓
                 │                                                            
{'1100': 100}

找出概率幅最大值的位置。

In [6]:
index = np.argmax(np.abs(qs))
print(index)

12


注意此时的 ``index`` 对应的 $x$ 并不是真正的估计值，被 $2^t$ 除之后也不是，因为测量结果中包括第二寄存器中的辅助比特，需要将``index``转成二进制后将辅助位剔除。

In [7]:
bit_string = bin(index)[2:].zfill(c.n_qubits)[1:]
print(bit_string)

100


在将二进制转回十进制，得到我们最终的估计值。

In [8]:
# pylint: disable=W0104
theta_exp = int(bit_string[::-1], 2) / 2**n
theta_exp

0.125

可见得到的估计相位和 $\varphi$ 近似相等。

## 参考文献

[1] Michael A. Nielsen and Isaac L. Chuang. [Quantum computation and quantum information](www.cambridge.org/9781107002173)